In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import simpy
import hyperopt

In [ ]:
dishes = pd.read_csv("dishes.csv")
dishes.head()

,ITEM,PRICE,FOOD TYPE,TIME,TYPE
0,FAST FARE BREAKFAST,10,BREAKFAST,5,0
1,EGGS BENEDICT,12,BREAKFAST,5,0
2,CINNAMON FRENCH TOAST,12,BREAKFAST,5,0
3,BREAKFAST TACOS,12,BREAKFAST,5,0
4,PANCAKES & EGGS,12,BREAKFAST,5,0


In [ ]:
orders = pd.read_csv('orders.csv')
orders = orders.applymap(str)
orders.head()

,Order_ID,TAKEAWAY/SEATING/PICKUP,DATE,DAY,TIME,SERVER,ITEM,QUANTITY,PRICE,TOTAL,ALLERGIES,FOOD TYPE,WEEK NUMBER
0,138908,TAKEAWAY,18-07-2020,FRIDAY,9:46,ASHLEY C.,FAST FARE BREAKFAST,1,10,10,NO,BREAKFAST,WEEK 1
1,138908,TAKEAWAY,18-07-2020,FRIDAY,9:46,ASHLEY C.,EGGS BENEDICT,1,12,12,NO,BREAKFAST,WEEK 1
2,138918,TAKEAWAY,18-07-2020,FRIDAY,10:03,ASHLEY C.,FAST FARE BREAKFAST,1,10,10,NO,BREAKFAST,WEEK 1
3,138923,TAKEAWAY,18-07-2020,FRIDAY,10:24,ASHLEY C.,BREAKFAST TACOS,1,12,12,NO,BREAKFAST,WEEK 1
4,138933,PICKUP,18-07-2020,FRIDAY,11:12,ASHLEY C.,PORK TENDERLOIN,2,22,44,YES,ENTRÉE,WEEK 1


In [ ]:
only_chief_skill = ['SPECIALS','PIZZA','RICE BOWLS AND PASTA','SANDWHICHES']


def order(env,chief,helper,Outcome):
    i = 0
    
    for i in range(len(Outcome['inter_time'])):
        yield env.timeout(Outcome['inter_time'][i])
        #print(i,'order',env.now)
        Outcome['start_time'].append(env.now)
        env.process(service(i,env,chief,helper,Outcome))
        i += 1

def service(i,env,chief,helper,Outcome):
    item = Outcome['dish'][i]
    process_t = int(dishes[dishes['ITEM']==item]['TIME'])*60
    type_ = dishes[dishes['ITEM']==item].iloc[0,2]
    pri = Outcome['priority'][item]
    preempt_c = True
    preempt_h = False
    rqtc = chief.request(priority=pri,preempt=preempt_c)
    rqth = helper.request(priority=pri,preempt=preempt_h)
    Event = rqth|rqtc
    result = yield Event
    while True:
        #print(i,'kaishi while true')
        try:
            if type_ not in only_chief_skill:
                if (rqth in result) and (rqtc not in result):
                    rqtc.cancel()
                    start = env.now
                    #print(i,'helper starts',env.now)
                    yield env.timeout(process_t*1.5)
                    helper.release(rqth) 
                    #print(i,'helper done',env.now)
                elif (rqtc in result) and (rqth not in result):
                    rqth.cancel()
                    start = env.now
                    #print(i,'chief starts',env.now)
                    yield env.timeout(process_t)
                    chief.release(rqtc)
                    #print(i,'chief done',env.now)
                else:
                    helper.release(rqth)
                    start = env.now
                    #print(i,'both can give chief first start',env.now)
                    yield env.timeout(process_t)
                    chief.release(rqtc)
                    #print(i,'both can give chief first done',env.now)
            else:
                #print('dao else')
                rqth.cancel()
                helper.release(rqth) 
                yield rqtc
                start = env.now
                #print(i,'only chief can do, starts',env.now)
                yield env.timeout(process_t)
                chief.release(rqtc)
                #print(i,'chief done',env.now)
            Outcome['end_time'].append(env.now)
            Outcome['revenue'].append(int(dishes[dishes['ITEM']==item]['PRICE']))
            break
        except simpy.Interrupt:
            #print('dao except')
            spent = env.now-start
            #rqtc.cancel()
            #chief.release(rqtc)
            pri = 0
            preempt_c = False
            process_t = process_t-spent
            rqtc = chief.request(priority=pri,preempt=preempt_c)
            rqth = helper.request(priority=pri,preempt=preempt_h)
            Event = rqth|rqtc
            result = yield Event

def priority_assign(w1,w2,w3,w4,w5):
    priority_ = {}
    for item in np.array(dishes['ITEM']):
        price = int(dishes[dishes['ITEM']==item]['PRICE'])
        process_t = int(dishes[dishes['ITEM']==item]['TIME'])
        type_ = int(dishes[dishes['ITEM']==item]['TYPE'])
        #priority_[item] = w1*price+w2*process_t+w3*type_
        if type_==0:            
            priority_[item] = w1*price+w2*process_t+w3
        elif type_==1:
            priority_[item] = w1*price+w2*process_t+w4
        else:
            priority_[item] = w1*price+w2*process_t+w5
    return priority_
    
    
def restaurant(f,priority_):
    order_time = []
    items = []
    times= np.array(f['TIME'])
    quantity = np.array(f['QUANTITY'])
    it = np.array(f['ITEM'])
    for i in range(f.shape[0]):
        t = int(times[i].split(':')[0])*60+int(times[i].split(':')[1])
        order_time.extend([t]*int(quantity[i]))
        items.extend([it[i]]*int(quantity[i]))
    order_time_sort = np.sort(order_time)
    index_sort = np.array(order_time).argsort()
    inter_time = [0]
    for i in range(len(order_time_sort)-1):
        inter = order_time_sort[i+1]-order_time_sort[i]
        inter_time.append(inter*60+1)
        
    env = simpy.Environment()
    chief = simpy.PreemptiveResource(env)
    helper = simpy.PreemptiveResource(env,capacity=2)
    
    dish = np.array(items)[index_sort]

    Outcome = {'inter_time':inter_time,'dish':dish,'priority':priority_,'start_time':[],'end_time':[],'revenue':[]}
    
    
    env.process(order(env,chief,helper,Outcome))
    env.run()
    
    waiting_time = [Outcome['end_time'][i]-Outcome['start_time'][i] for i in range(len(Outcome['end_time']))]
    p_w = [Outcome['revenue'][i]/waiting_time[i] for i in range(len(waiting_time))]
    #print('waiting_time',np.mean(waiting_time))
    #print('price/waiting_time',np.mean(p_w))
    return np.mean(waiting_time),np.mean(p_w)

In [ ]:
priority_ = priority_assign(1,2,1,1,1)
restaurant(orders[orders['DATE']== '19-07-2020'],priority_)

(524.4230769230769, 0.03279691937652518)

In [ ]:
def final(w1,w2,w3,w4,w5):
    waiting_times = []
    p_ws = []
    priority_ = priority_assign(w1,w2,w3,w4,w5)
    for date in set(set(orders['DATE'])):
        #print(date)
        f = orders[orders['DATE']== date] 
        waiting_times.append(restaurant(f,priority_)[0])
        p_ws.append(restaurant(f,priority_)[1])

    #return np.mean(waiting_times)
    return np.mean(p_ws)
    #print('Average waiting time:', np.mean(waiting_times))
    #print('Price/waiting time:', np.mean(p_ws))

In [ ]:
def obj(params):
    a=params[0]
    b=params[1]
    c = params[2]
    d = params[3]
    e = params[4]
    diffs=final(a,b,c,d,e)
    return -1*diffs
  
trials=hyperopt.Trials()
best = hyperopt.fmin(fn=obj,
    space=[hyperopt.hp.uniform('a', -10, 10),hyperopt.hp.uniform('b', -10, 10),hyperopt.hp.uniform('c', -10, 10),hyperopt.hp.uniform('d', -10, 10),hyperopt.hp.uniform('e', -10, 10)],
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    #timeout=1,
    trials=trials)

print(best)

100%|██████████| 50/50 [05:37<00:00,  6.75s/trial, best loss: -0.024221497120780908]
{'a': -0.438030253853003, 'b': 2.383215020358225, 'c': 9.933321379510911, 'd': -9.65526897265401, 'e': 2.9052279341355245}


In [ ]:
#Baseline Result without priority
Base = final(0,0,0,0,0)

##Optimized Result
Optimized = final(-0.438030253853003,2.383215020358225,9.933321379510911,-9.65526897265401,2.9052279341355245)

print("The baseline result for price/unit is:",Base)
print("The optimized result for price/unit is:",Optimized)
print("The efficiency is increased:",(Optimized-Base)/Base*100)

The baseline result for price/unit is: 0.023491626808409893
The optimized result for price/unit is: 0.024221497120780908
The efficiency is increased: 3.1069381372503537


In [ ]:
0.02439/0.0235#1.5倍数，w1*price+w2*process_t+w3

1.037872340425532